# Phase 1: Build PnL Time Series per Cluster

This notebook contains the analysis for the Risk & Portfolio Interaction study.
Reminder: Belkhayat Quuick call tommorrow 4 PM for strategy phases comfirmation 


Phase 1: Build PnL Time Series per Cluster
==========================================
Goal: Turn trades into time series for portfolio-level risk analysis

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("=" * 80)
print("PHASE 1: BUILD PnL TIME SERIES PER CLUSTER")
print("=" * 80)

# Load the main trades data
trades_df = pd.read_csv('trades_with_clusters.csv')

# Convert datetime columns
trades_df['entry_time'] = pd.to_datetime(trades_df['entry_time'])
trades_df['exit_time'] = pd.to_datetime(trades_df['exit_time'])

print(f"\n Dataset Overview:")
print(f"   Total trades: {len(trades_df)}")
print(f"   Date range: {trades_df['entry_time'].min().date()} to {trades_df['exit_time'].max().date()}")
print(f"   Clusters: {sorted(trades_df['cluster'].unique())}")
print(f"   Total profit: ${trades_df['profit'].sum():.2f}")

# Cluster distribution
print(f"\n Trades per Cluster:")
for cluster in sorted(trades_df['cluster'].unique()):
    cluster_trades = trades_df[trades_df['cluster'] == cluster]
    print(f"   Cluster {int(cluster)}: {len(cluster_trades)} trades, Total PnL: ${cluster_trades['profit'].sum():.2f}")

# Step 1: Create date column from exit_time
trades_df['date'] = trades_df['exit_time'].dt.floor('D')

# Step 2: Group by date and cluster, sum profit
daily_pnl = trades_df.groupby(['date', 'cluster'])['profit'].sum().reset_index()

print(f"\n Daily PnL Summary:")
print(f"   Trading days: {daily_pnl['date'].nunique()}")
print(f"   Daily entries: {len(daily_pnl)}")

# Step 3: Pivot to get one column per cluster
daily_pnl_pivot = daily_pnl.pivot(index='date', columns='cluster', values='profit')
daily_pnl_pivot.columns = [f'cluster_{int(c)}_pnl' for c in daily_pnl_pivot.columns]
daily_pnl_pivot = daily_pnl_pivot.fillna(0)  # Days without trades = 0 PnL

print(f"\n Daily PnL Matrix Shape: {daily_pnl_pivot.shape}")
print(f"   Columns: {list(daily_pnl_pivot.columns)}")

# Create total portfolio PnL
daily_pnl_pivot['total_pnl'] = daily_pnl_pivot.sum(axis=1)

# Cumulative PnL
cumulative_pnl = daily_pnl_pivot.cumsum()

print("\n" + "=" * 80)
print("DAILY PnL STATISTICS")
print("=" * 80)
print(daily_pnl_pivot.describe().round(2))

# Save to CSV for next phases
daily_pnl_pivot.to_csv('daily_pnl_per_cluster.csv')
print(f"\n Daily PnL data saved to: daily_pnl_per_cluster.csv")

PHASE 1: BUILD PnL TIME SERIES PER CLUSTER

 Dataset Overview:
   Total trades: 430
   Date range: 2025-01-07 to 2025-11-20
   Clusters: [0.0, 1.0, 2.0, 3.0]
   Total profit: $4449.42

 Trades per Cluster:
   Cluster 0: 200 trades, Total PnL: $2318.39
   Cluster 1: 93 trades, Total PnL: $1630.88
   Cluster 2: 76 trades, Total PnL: $298.43
   Cluster 3: 61 trades, Total PnL: $201.72

 Daily PnL Summary:
   Trading days: 167
   Daily entries: 271

 Daily PnL Matrix Shape: (167, 4)
   Columns: ['cluster_0_pnl', 'cluster_1_pnl', 'cluster_2_pnl', 'cluster_3_pnl']

DAILY PnL STATISTICS
       cluster_0_pnl  cluster_1_pnl  cluster_2_pnl  cluster_3_pnl  total_pnl
count         167.00         167.00         167.00         167.00     167.00
mean           13.88           9.77           1.79           1.21      26.64
std            38.21          28.72          25.64          17.93      61.93
min          -153.20         -60.55        -149.67        -100.41    -177.88
25%             0.00        

In [ ]:
# VISUALIZATIONS

In [5]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Cumulative PnL by Cluster
ax1 = axes[0, 0]
for col in [c for c in cumulative_pnl.columns if c != 'total_pnl']:
    ax1.plot(cumulative_pnl.index, cumulative_pnl[col], label=col.replace('_pnl', ''), linewidth=2)
ax1.set_title('Cumulative PnL by Cluster', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Cumulative PnL ($)')
ax1.legend(loc='upper left')
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax1.grid(True, alpha=0.3)

# Plot 2: Total Portfolio Cumulative PnL
ax2 = axes[0, 1]
ax2.fill_between(cumulative_pnl.index, 0, cumulative_pnl['total_pnl'], 
                  where=cumulative_pnl['total_pnl']>=0, color='green', alpha=0.3)
ax2.fill_between(cumulative_pnl.index, 0, cumulative_pnl['total_pnl'], 
                  where=cumulative_pnl['total_pnl']<0, color='red', alpha=0.3)
ax2.plot(cumulative_pnl.index, cumulative_pnl['total_pnl'], color='blue', linewidth=2)
ax2.set_title('Total Portfolio Cumulative PnL', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('Cumulative PnL ($)')
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax2.grid(True, alpha=0.3)

# Plot 3: Daily PnL Distribution by Cluster
ax3 = axes[1, 0]
cluster_cols = [c for c in daily_pnl_pivot.columns if c != 'total_pnl']
daily_pnl_melted = daily_pnl_pivot[cluster_cols].melt(var_name='Cluster', value_name='Daily PnL')
sns.boxplot(data=daily_pnl_melted, x='Cluster', y='Daily PnL', ax=ax3)
ax3.set_title('Daily PnL Distribution by Cluster', fontsize=14, fontweight='bold')
ax3.set_xticklabels([c.replace('_pnl', '') for c in cluster_cols], rotation=45)
ax3.axhline(y=0, color='red', linestyle='--', alpha=0.5)
ax3.grid(True, alpha=0.3)

# Plot 4: Stacked Area - Contribution to Portfolio PnL
ax4 = axes[1, 1]
positive_pnl = daily_pnl_pivot[cluster_cols].clip(lower=0)
negative_pnl = daily_pnl_pivot[cluster_cols].clip(upper=0)
ax4.stackplot(daily_pnl_pivot.index, 
              [positive_pnl[c] for c in cluster_cols],
              labels=[c.replace('_pnl', '') for c in cluster_cols], alpha=0.7)
ax4.set_title('Daily Positive PnL Contribution by Cluster', fontsize=14, fontweight='bold')
ax4.set_xlabel('Date')
ax4.set_ylabel('Daily PnL ($)')
ax4.legend(loc='upper left')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('phase1_pnl_timeseries.png', dpi=150, bbox_inches='tight')
plt.close()
print(f" Phase 1 visualization saved to: phase1_pnl_timeseries.png")

 Phase 1 visualization saved to: phase1_pnl_timeseries.png


In [ ]:
# SUMMARY FOR REPORT

In [6]:
print("\n" + "=" * 80)
print("PHASE 1 SUMMARY")
print("=" * 80)
summary = {
    'Total Trades': len(trades_df),
    'Date Range': f"{trades_df['entry_time'].min().date()} to {trades_df['exit_time'].max().date()}",
    'Trading Days': daily_pnl_pivot.shape[0],
    'Total Portfolio PnL': f"${trades_df['profit'].sum():.2f}"
}
for k, v in summary.items():
    print(f"   {k}: {v}")

# Per cluster stats
print("\n   PnL per Cluster:")
for cluster in sorted(trades_df['cluster'].unique()):
    cluster_data = trades_df[trades_df['cluster'] == cluster]
    print(f"      Cluster {int(cluster)}: {len(cluster_data)} trades, ${cluster_data['profit'].sum():.2f}")

print("\n PHASE 1 COMPLETED AND PUSHED TO GIT ")


PHASE 1 SUMMARY
   Total Trades: 430
   Date Range: 2025-01-07 to 2025-11-20
   Trading Days: 167
   Total Portfolio PnL: $4449.42

   PnL per Cluster:
      Cluster 0: 200 trades, $2318.39
      Cluster 1: 93 trades, $1630.88
      Cluster 2: 76 trades, $298.43
      Cluster 3: 61 trades, $201.72

 PHASE 1 COMPLETED AND PUSHED TO GIT 
